In [10]:
import json

with open('/home/mle-user/sprints/mle-project-sprint-1-v001/part2_dvc/cv_results/cv_res.json') as json_file:
    results = json.load(json_file)

metrics = {}
for x in results.split('   '):
    metric = x.split(':')
    metrics[metric[0]] = float(metric[1].strip())
metrics['R2'] = metrics.pop('R^2')    

In [11]:
import pandas as pd

train_data = pd.read_csv('/home/mle-user/sprints/mle-project-sprint-1-v001/part2_dvc/data/train_data.csv')

X_train = train_data.drop(columns=['price'])
y_train = train_data['price']

test_data = pd.read_csv('/home/mle-user/sprints/mle-project-sprint-1-v001/part2_dvc/data/test_data.csv')

X_test = test_data.drop(columns=['price'])
y_test = test_data['price']

In [12]:
import joblib

with open('/home/mle-user/sprints/mle-project-sprint-1-v001/part2_dvc/models/fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd) 

/home/mle-user/sprints/mle-project-sprint-2-v001/.venv_sprint2/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.4.1.post1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
prediction = model.predict(X_test)

In [17]:
import mlflow
import os

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") 
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") 


TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "sprint_2"
RUN_NAME = "baseline_model"

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

pip_requirements = "requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metrics(metrics)
    model_info = mlflow.sklearn.log_model( 
		sk_model=model, artifact_path='models', registered_model_name='baseline_model', pip_requirements=pip_requirements, signature=signature, input_example=input_example, metadata=metadata, await_registration_for=60
		)

Successfully registered model 'baseline_model'.
2024/10/05 16:00:58 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: baseline_model, version 1
Created version '1' of model 'baseline_model'.
